In [2]:
!pip install -q Kaggler

     |████████████████████████████████| 829kB 13.6MB/s 


In [28]:
import os
import glob
import pickle
import numpy as np
import pandas as pd
import warnings

from sklearn.metrics import log_loss
from sklearn.model_selection import StratifiedKFold, train_test_split, cross_val_score
import tensorflow as tf

from kaggler.preprocessing import LabelEncoder
from kaggler.model import AutoLGB
from xgboost import XGBClassifier
from hyperopt import STATUS_OK, Trials, fmin, hp, tpe
import kaggler

In [29]:
index_col = 'index'
target_col = 'credit'

X = pd.read_csv('train.csv', index_col=index_col, encoding='utf-8').drop(columns=['credit'])
y = pd.read_csv('train.csv', index_col=index_col, encoding='utf-8')['credit']
X_test = pd.read_csv('test.csv', index_col=index_col, encoding='utf-8')

print(f'Train Data Shape: {X.shape}\nTest Data Shape: {X_test.shape}')

Train Data Shape: (26457, 18)
Test Data Shape: (10000, 18)


In [30]:
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.3, random_state=42)

print(X_train.shape, X_valid.shape)

(18519, 18) (7938, 18)


In [31]:
cat_features = [i for i in X.columns if X[i].dtype == 'object']
num_features = [i for i in X.columns if i not in cat_features + [target_col]]
features = num_features + cat_features

print(f'The Number of Categorical Features: {len(cat_features)}\nThe Number of Numeric Features: {len(num_features)}\nThe Number of Total Features: {len(features)}')

The Number of Categorical Features: 8
The Number of Numeric Features: 10
The Number of Total Features: 18


In [32]:
lde = LabelEncoder(min_obs=10)

X_train[cat_features] = lde.fit_transform(X_train[cat_features])
X_valid[cat_features] = lde.transform(X_valid[cat_features])
X_test[cat_features] = lde.transform(X_test[cat_features])

/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1781: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item_labels[indexer[info_axis]]] = value
/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1743: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(ilocs[0], value)
/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1743: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = valu

In [33]:
encoding_dim = 64

def get_model(encoding_dim, dropout=0.2):
    num_dim = len(num_features)
    num_input = tf.keras.layers.Input((num_dim,), name='num_input')
    cat_inputs = []
    cat_embs = []
    emb_dims = 0
    for col in cat_features:
        cat_input = tf.keras.layers.Input((1,), name=f'{col}_input')
        emb_dim = max(8, int(np.log2(1 + X_train[col].nunique()) * 4))
        cat_emb = tf.keras.layers.Embedding(input_dim=int(X_train[col].max()) + 1, output_dim=emb_dim)(cat_input)
        cat_emb = tf.keras.layers.Dropout(dropout)(cat_emb)
        cat_emb = tf.keras.layers.Reshape((emb_dim,))(cat_emb)

        cat_inputs.append(cat_input)
        cat_embs.append(cat_emb)
        emb_dims += emb_dim

    merged_inputs = tf.keras.layers.Concatenate()([num_input] + cat_embs)

    encoded = tf.keras.layers.Dense(encoding_dim * 3, activation='relu')(merged_inputs)
    encoded = tf.keras.layers.Dropout(dropout)(encoded)
    encoded = tf.keras.layers.Dense(encoding_dim * 2, activation='relu')(encoded)
    encoded = tf.keras.layers.Dropout(dropout)(encoded)    
    encoded = tf.keras.layers.Dense(encoding_dim, activation='relu')(encoded)
    
    decoded = tf.keras.layers.Dense(encoding_dim * 2, activation='relu')(encoded)
    decoded = tf.keras.layers.Dropout(dropout)(decoded)
    decoded = tf.keras.layers.Dense(encoding_dim * 3, activation='relu')(decoded)
    decoded = tf.keras.layers.Dropout(dropout)(decoded)    
    decoded = tf.keras.layers.Dense(num_dim + emb_dims, activation='linear')(encoded)

    encoder = tf.keras.Model([num_input] + cat_inputs, encoded)
    ae = tf.keras.Model([num_input] + cat_inputs, decoded)
    ae.add_loss(tf.keras.losses.mean_squared_error(merged_inputs, decoded))
    ae.compile(optimizer='adam')

    return ae, encoder

ae, encoder = get_model(encoding_dim)
ae.summary()

Model: "model_5"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
gender_input (InputLayer)       [(None, 1)]          0                                            
__________________________________________________________________________________________________
car_input (InputLayer)          [(None, 1)]          0                                            
__________________________________________________________________________________________________
reality_input (InputLayer)      [(None, 1)]          0                                            
__________________________________________________________________________________________________
income_type_input (InputLayer)  [(None, 1)]          0                                            
____________________________________________________________________________________________

In [34]:
%%time

inputs = [X_train[num_features].values] + [X_train[i].values for i in cat_features]

ae.fit(inputs, inputs,
       epochs=100,
       batch_size=16384,
       shuffle=True,
       validation_split=0.2)

Epoch 1/100
1/1 [==============================] - 2s 2s/step - loss: 826332800.0000 - val_loss: 750382976.0000
Epoch 2/100
1/1 [==============================] - 0s 77ms/step - loss: 775452096.0000 - val_loss: 716517056.0000
Epoch 3/100
1/1 [==============================] - 0s 74ms/step - loss: 736558144.0000 - val_loss: 690145024.0000
Epoch 4/100
1/1 [==============================] - 0s 72ms/step - loss: 704710592.0000 - val_loss: 668436992.0000
Epoch 5/100
1/1 [==============================] - 0s 74ms/step - loss: 678923904.0000 - val_loss: 648667456.0000
Epoch 6/100
1/1 [==============================] - 0s 77ms/step - loss: 656338240.0000 - val_loss: 629012288.0000
Epoch 7/100
1/1 [==============================] - 0s 76ms/step - loss: 635182976.0000 - val_loss: 609877952.0000
Epoch 8/100
1/1 [==============================] - 0s 74ms/step - loss: 615788096.0000 - val_loss: 591292032.0000
Epoch 9/100
1/1 [==============================] - 0s 82ms/step - loss: 597640768.0000 - v

In [38]:
features

['child_num',
 'income_total',
 'DAYS_BIRTH',
 'DAYS_EMPLOYED',
 'FLAG_MOBIL',
 'work_phone',
 'phone',
 'email',
 'family_size',
 'begin_month',
 'gender',
 'car',
 'reality',
 'income_type',
 'edu_type',
 'family_type',
 'house_type',
 'occyp_type']

In [50]:
def feature_ae(dataset):
  inputs = [dataset[num_features].values] + [dataset[i].values for i in cat_features]
  encoding = encoder.predict(inputs)
  encoded = pd.concat([dataset[features], pd.DataFrame(encoding, columns=[f'enc_{i}' for i in range(encoding_dim)])], axis=1).reindex(dataset[features].index)

  return encoded

X_train_encoded = feature_ae(X_train)
X_valid_encoded = feature_ae(X_valid)
X_test_encoded = feature_ae(X_test)

In [51]:
X_train_encoded.shape, X_valid_encoded.shape, X_test_encoded.shape

((18519, 82), (7938, 82), (10000, 82))

In [52]:
X_train_encoded

,child_num,income_total,DAYS_BIRTH,DAYS_EMPLOYED,FLAG_MOBIL,work_phone,phone,email,family_size,begin_month,gender,car,reality,income_type,edu_type,family_type,house_type,occyp_type,enc_0,enc_1,enc_2,enc_3,enc_4,enc_5,enc_6,enc_7,enc_8,enc_9,enc_10,enc_11,enc_12,enc_13,enc_14,enc_15,enc_16,enc_17,enc_18,enc_19,enc_20,enc_21,...,enc_24,enc_25,enc_26,enc_27,enc_28,enc_29,enc_30,enc_31,enc_32,enc_33,enc_34,enc_35,enc_36,enc_37,enc_38,enc_39,enc_40,enc_41,enc_42,enc_43,enc_44,enc_45,enc_46,enc_47,enc_48,enc_49,enc_50,enc_51,enc_52,enc_53,enc_54,enc_55,enc_56,enc_57,enc_58,enc_59,enc_60,enc_61,enc_62,enc_63
index,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1086,0.0,67500.0,-20900.0,365243.0,1.0,0.0,1.0,0.0,2.0,-11.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,50139.300781,26448.746094,66764.523438,0.0,0.0,79507.250000,0.0,0.0,0.0,0.0,30547.861328,61239.230469,0.0,50772.285156,29191.375000,0.0,37276.191406,0.0,0.0,...,0.0,30118.466797,0.0,0.0,44514.304688,42751.816406,0.0,0.0,0.0,0.0,11386.277344,0.0,55735.402344,0.0,40014.980469,0.0,0.0,0.0,0.0,64791.933594,0.0,77837.039062,49872.828125,25089.023438,39540.691406,0.0,0.0,0.0,0.0,65773.015625,0.0,0.0,0.0,106505.906250,51822.476562,0.0,0.0,0.0,54024.960938,0.0
2993,0.0,180000.0,-19408.0,-4553.0,1.0,0.0,1.0,0.0,2.0,-58.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,31597.000000,15440.605469,41550.953125,0.0,0.0,49190.222656,0.0,0.0,0.0,0.0,17968.007812,37227.332031,0.0,29968.673828,17506.375000,0.0,22887.421875,0.0,0.0,...,0.0,18107.826172,0.0,0.0,26888.234375,25266.210938,0.0,0.0,0.0,0.0,6940.037598,0.0,33456.144531,0.0,23746.417969,0.0,0.0,0.0,0.0,39448.160156,0.0,47440.824219,30989.353516,15758.159180,24412.089844,0.0,0.0,0.0,0.0,41315.796875,0.0,0.0,0.0,65505.957031,31649.472656,0.0,0.0,0.0,32475.582031,0.0
16219,0.0,180000.0,-14279.0,-2939.0,1.0,0.0,0.0,0.0,1.0,-39.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,3.0,0.0,0.0,0.0,60650.476562,32128.652344,80745.515625,0.0,0.0,96225.046875,0.0,0.0,0.0,0.0,37040.703125,74189.468750,0.0,61481.082031,35387.519531,0.0,45016.871094,0.0,0.0,...,0.0,36403.484375,0.0,0.0,53830.964844,51831.136719,0.0,0.0,0.0,0.0,13799.448242,0.0,67486.031250,0.0,48402.285156,0.0,0.0,0.0,0.0,78380.257812,0.0,94152.039062,60357.527344,30275.152344,47942.917969,0.0,0.0,0.0,0.0,79497.679688,0.0,0.0,0.0,128844.343750,62731.054688,0.0,0.0,0.0,65416.269531,0.0
14707,0.0,157500.0,-13783.0,-2058.0,1.0,0.0,0.0,1.0,2.0,-30.0,0.0,1.0,0.0,4.0,1.0,2.0,0.0,2.0,0.0,0.0,0.0,37461.480469,18860.722656,49524.113281,0.0,0.0,58718.667969,0.0,0.0,0.0,0.0,21888.298828,44716.652344,0.0,36465.222656,21165.128906,0.0,27442.574219,0.0,0.0,...,0.0,21886.380859,0.0,0.0,32449.472656,30719.328125,0.0,0.0,0.0,0.0,8320.559570,0.0,40404.351562,0.0,28838.593750,0.0,0.0,0.0,0.0,47415.082031,0.0,57009.941406,36926.886719,18753.519531,29126.861328,0.0,0.0,0.0,0.0,49041.078125,0.0,0.0,0.0,78420.382812,37980.617188,0.0,0.0,0.0,39222.664062,0.0
4166,0.0,225000.0,-17915.0,-3366.0,1.0,0.0,0.0,0.0,2.0,-44.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,69032.921875,35564.640625,91508.187500,0.0,0.0,108631.609375,0.0,0.0,0.0,0.0,41089.550781,83093.703125,0.0,68215.617188,39550.027344,0.0,50726.832031,0.0,0.0,...,0.0,40659.003906,0.0,0.0,60366.109375,57575.820312,0.0,0.0,0.0,0.0,15408.041016,0.0,75233.148438,0.0,53787.160156,0.0,0.0,0.0,0.0,88002.882812,0.0,105789.101562,68235.460938,34548.765625,53991.824219,0.0,0.0,0.0,0.0,90238.765625,0.0,0.0,0.0,145251.000000,70510.898438,0.0,0.0,0.0,73104.070312,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21575,3.0,135000.0,-14838.0,-1539.0,1.0,0.0,0.0,0.0,5.0,-47.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,

In [53]:
clf_xgb = XGBClassifier(verbosity=0, objective='multi:softprob', eval_metric='mlogloss')
clf_xgb.fit(X_train_encoded, y_train, verbose=False)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, eval_metric='mlogloss',
              gamma=0, learning_rate=0.1, max_delta_step=0, max_depth=3,
              min_child_weight=1, missing=None, n_estimators=100, n_jobs=1,
              nthread=None, objective='multi:softprob', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=1, verbosity=0)

In [54]:
y_pred = clf_xgb.predict_proba(X_valid_encoded)
log_loss(y_true=y_valid, y_pred=y_pred)

0.8076105017208444

In [70]:
y_pred = clf_xgb.predict_proba(X_test_encoded)
submission = pd.DataFrame(y_pred)
submission['index'] = X_test.index
submission = submission[['index', 0, 1, 2]]
submission.to_csv('submission_0429_01.csv', encoding='utf-8', index=False)

In [ ]:
%%time

space = {
    'learning_rate': hp.quniform('learning_rate', 0.01, 0.1, 0.01)
}


def objective(space):
    clf_xgb = XGBClassifier(learning_rate=space['learning_rate'],
                            verbosity=0,
                            objective='multi:softprob',
                            eval_metric='mlogloss')
    
    clf_xgb.fit(X_train_encoded, y_train, early_stopping_rounds=10, eval_set=[(X_valid_encoded, y_valid)], verbose=False)
    scores = cross_val_score(estimator=clf_xgb, X=X_train_encoded, y=y_train, cv=5, scoring='neg_log_loss')
    cv_mean = scores.mean()
    
    return {'loss': 1 - cv_mean, 'status': STATUS_OK}

trials = Trials()
best = fmin(fn=objective,
            space=space,
            algo=tpe.suggest,
            max_evals=30,
            trials=trials)

 17%|█▋        | 5/30 [06:14<31:09, 74.80s/it, best loss: 1.8009957753667325]